In [1]:
import os
import sys
import fasttext
import pandas as pd
import matplotlib.pyplot as plt
import neattext.functions as nfx
import requests

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from code_utils.enriching_data_OpenAlex import bool_topics


ModuleNotFoundError: No module named 'code_utils'

In [ ]:
climat_topics=['climate change','marine ecosystems','ecology','ecological','ecosystem','environmental','global methane emissions and impacts','impact of ocean acidification on marine ecosystems','arctic sea ice variability and decline','environmental impact','climate ethics','climate and hydrological cycle','global energy transition','environmental behavior','influence of climate','urban heat islands and mitigation strategies','impact on climate','environmental policies','carbon dioxide capture and storage technologies','soil carbon dynamics and nutrient cycling in ecosystems','sustainable development','environmental governance','atmospheric aerosols and their impacts','marine biogeochemistry and ecosystem dynamics','biodiversity','global flood risk assessment and management','arctic sea ice variability and decline','dynamics of ocean surface waves and wind interaction','aeolian geomorphology and wind erosion dynamics','importance of mangrove ecosystems in coastal protection','water resource management','air pollution','deforestation','rebound effect on energy efficiency and consumption','global drought','landslide hazards and risk assessment','paleoclima']


In [ ]:
url=f"https://api.openalex.org/works?filter=has_doi:true,topics_count:>0,publication_year:<2024,publication_year:>2018,institutions.country_code:{country}&sample=200&per-page=200"
response = requests.get(url)
data0 = response.json().get('results')